# Spectral Density Estimation
This notebook contains a demonstration of three different Power Spectrum estimation techniques.  The first two are classical techniques (periodogram and Blackman and Tukey), and the third is a modern non-parametric technique (minimum variance spectral estimation, MVSE).

### Preamble
Start by importing the Python libraries that we will require

In [ ]:
import pylab as pl
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import scipy.fftpack as spf
import scipy.linalg as spl
from IPython import display

And define a function that will return true if running in a Jupyter Notebook

In [ ]:
def is_jupyter():
    """Return true if running in a Jupyter Notebook"""
    try:
        if get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
            return True
        else:
            return False
    except: 
        return False

### User specified parameters

The following parameters can be specified.  

Parameter | Meaning
--------- | -------
<code>p</code> | The number of terms in the autocorrelation for the minimum variance spectral estimate (e.g. 256)
<code>M</code> | The block size for the periodogram approaches, and maximum delay for the correlogram (e.g. 256)
<code>fs</code> | Length of the FFT after zero padding the data blocks (e.g. 2048)
<code>f1</code> | Frequency of first tone
<code>f2</code> | Frequency of second tone

In [ ]:
p = 256
M = 256
fs = 2048

f1 = 99.5 / M
f2 = 104 / M

### The signal to be analysed

In [ ]:
n = np.arange(0, 100001)
x = np.sin(2*np.pi*n*f1) + 0.5*np.sin(2*np.pi*n*f2) + 0.2*np.random.randn(n.size)

# For plotting of figures
frequency = np.arange(0, fs) / fs

### Function Definitions
First we define an efficient autocorrelation function that returns the autocorrelation for a limited set of delays.  The algorithm that it implements is described in the lecture notes.

In [ ]:
def Autocorrelation(x, M):
    """
        Calculate autocorrelation of x with maximum lag M.
        
        INPUT:
            x - vector to be correlated
            M - maximum correlation lag
        
        RETURN:
            acf - autocorrelation of x with lag M
        
    """

    #### Step 1 - initialise the index to identify the data block
    #### We don't need to copy the data block as Matlab can select
    #### the block at the call time of the FFT.  i is reserved for
    #### sqrt(-1), so call it index instead
    index = 0
    
    #### Step 2 - compute the FFT of x_i(n)
    X = sp.fftpack.fft(x[index*(M+1):(index+1)*(M+1)], 2*M+2)

    #### Steps 3 to 6 are repeated, accumulating their results in
    #### a vector.  We need to initialise the vector first

    result = np.zeros(2*M+2)

    # It is also helpful to generate the vector of [ 1 -1 1 ... ]

    phase_shift = np.power((-1), np.arange(0, 2*M+2))

    # Do the repetition until we run out of data
    while (True):

        #### Step 3 - compute X_i(k)X^*_i(k) and store

        result = result + np.multiply(np.conj(X), X).real

        #### Step 4 - increment i

        index = index + 1

        # Check to see if we have used all of the data
        if (index*(M+1) > len(x)):
            break

        #### Step 5 - compute the transform for the next block

        if ((index+1)*(M+1) <= len(x)):
            nextX = sp.fftpack.fft(x[index*(M+1):(index+1)*(M+1)], 2*M+2)
        else:
            # We don't always have a full block of data at the end
            # of the record, but we still need to process it
            nextX = sp.fftpack.fft(x[index*(M+1):], 2*M+2)

        #### Step 6 - add in the product of the previous and next
        ####          transforms, with the phase shift

        result = result + np.multiply(np.multiply(phase_shift, np.conj(X)), nextX)

        #### Step 7 - repeat steps 3 to 6 until all of the data
        #### has been used.  Before we do this, we need
        #### to make X = nextX

        X = nextX

    #### Step 8 - inverse FFT

    time_domain = sp.fftpack.ifft(result, 2*M+2)

    #### Step 9 = present only the first M+1 values

    acf = np.divide(time_domain[0:M+1], len(x)).real
    
    return acf

Now we define a function to produce the plots that we need.

In [ ]:
def generate_plot(x, y,
                  xlabel='Normalised frequency',
                  ylabel='Magnitude (dB)',
                  xlim=[0,0.5], ylim=[-20,35], name=''):
    """
       Plot the magnitude of normalised freuqnecy
       INPUT:
           x  (array-like): The horizontal coordinates of the data points.
           y  (array-like): The vertical coordinates of the data points.
           xlabel (string): Label for the x-axis
           ylabel (string): Label for the y-axis
           name   (string): The name used to save figure.
    """
    # Create the plot figure
    plt.figure(figsize = (16, 8))
    
    # Enlarge figure label and axis size
    plt.rcParams.update({'font.size': 16})
    
    # Plot the frequency
    plt.plot(x, y)
    
    # Tidy up the plot to control axes sizes and labels
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xlim(xlim)
    plt.ylim(ylim)
    #plt.xticks(np.linspace(0, 0.1, 11))
    
    # Save figure in python or ipython system
    if not is_jupyter():
        plt.tight_layout()
        plt.savefig(name)

### Plot input sample
Plot a sample of the input signal

In [ ]:
generate_plot(x=n[:fs+1], y=x[:fs+1], xlabel='Sample', ylabel='Amplitude',
              xlim=[0,fs], ylim=[-2,2], name='Random_signal_sample.pdf')

## Periodograms
The first technique we will consider are the periodogram approaches.  The basic periodogram computes $$P_{xx}(k) = \frac{1}{M}\left|\sum_{n=0}^{M-1} x(n)e^{-j 2 \pi k n/M}\right|^2$$

In [ ]:
# Select the first M elements and compute the periodogram
Px = 20*np.log10(abs(np.fft.fft(x[:M],fs)))-10*np.log10(M)
generate_plot(x=frequency, y=Px, name='Periodogram_estimate_1.pdf')

# Repeat this for the next M elements
Px = 20*np.log10(abs(np.fft.fft(x[M:2*M],fs)))-10*np.log10(M)
generate_plot(x=frequency, y=Px, name='Periodogram_estimate_2.pdf')

### Bartlett periodogram
The Bartlett periodogram simply averages a set of periodograms like those produced above.  It is written as:
$$ P_{xx}^{\mathrm{B}}(k) = \frac{1}{K}\sum_{i=0}^{K-1}{\left\{\frac{1}{M}\left|\sum_{n=0}^{M-1}{x(n+iM)e^{-j2\pi k n/M}}\right|^2\right\}}$$

In [ ]:
# Find out how many periodograms we can compute
K = np.floor(len(x)/M).astype(int)

# Initialise the accumulation of periodograms
Px = np.zeros(fs)

for idx in range(K):
    Px = Px + np.power(abs(np.fft.fft(x[idx*M:(idx+1)*M],fs)),2)/M

# Convert to dB
Px = 10 * np.log10(Px) - 10*np.log10(K)

generate_plot(x=frequency, y=Px, name='Bartlett_periodogram_estimate.pdf')

### Welch periodogram
The Welch method is a modification of the Bartlett method.  Each block of data is windowed prior to computing the discrete Fourier transform, and the blocks of taken from the input are overlapping.  Mathematically:
$$P_{xx}^{\mathrm{W}}(k)\!=\! \frac{1}{L}\!\sum_{i=0}^{L-1}\!{\left\{\!\frac{1}{MU}\left|\sum_{n=0}^{M-1}{x(n+iD)w(n)e^{-j2\pi k n/M}}\right|^2\!\right\}}$$
where $w(n)$ is the window, and $M-D$ is the number of sampes that overlap from one block to the next.  We will start with a 50% overlap, and apply the Hamming window.

In [ ]:
D = M // 2

# Find out how many periodograms we can compute
L = np.floor((len(x)-M)/D + 1).astype(int)

# Define the window
w = 0.54 - 0.46 * np.cos(2 * np.pi * np.arange(0,M) / (M-1))

# Calculate the power of the window
U = np.dot(w,w)/M

# Initialise the accumulation of periodograms
Px = np.zeros(fs)

for idx in range(L):
    Px = Px + np.power(abs(np.fft.fft(np.multiply(w,x[idx*D:idx*D+M]),fs)),2)/(M*U)

# Convert to dB
Px = 10 * np.log10(Px) - 10*np.log10(L)

generate_plot(x=frequency, y=Px, name='Welch_periodogram_Hamming.pdf')

Alternative windows can also be used, for example, the Hann window.  In this case, the peak sidelobe level is higher than for the Hamming window.

In [ ]:
# Define the window
w = 0.5 - 0.5 * np.cos(2 * np.pi * np.arange(0,M) / (M-1))

# Calculate the power of the window
U = np.dot(w,w)/M

# Initialise the accumulation of periodograms
Px = np.zeros(fs)

for idx in range(L):
    Px = Px + np.power(abs(np.fft.fft(np.multiply(w,x[idx*D:idx*D+M]),fs)),2)/(M*U)

# Convert to dB
Px = 10 * np.log10(Px) - 10*np.log10(L)

generate_plot(x=frequency, y=Px, name='Welch_periodogram_Hann.pdf')

Repeating for the Blackman window, the wider main lobe is evident in the resulting plot.  The advantage of this window is the much lower sidelobe levels, but with the level of noise in the signal, there is no benefit in using this window instead of the Hamming window.  In fact, the wider main lobe makes this window a worse choice for this particular signal.

In [ ]:
# Define the window
w = 0.42 - 0.5 * np.cos(2 * np.pi * np.arange(0,M) / (M-1)) \
        + 0.08 * np.cos(4 * np.pi * np.arange(0,M) / (M-1))

# Calculate the power of the window
U = np.dot(w,w)/M

# Initialise the accumulation of periodograms
Px = np.zeros(fs)

for idx in range(L):
    Px = Px + np.power(abs(np.fft.fft(np.multiply(w,x[idx*D:idx*D+M]),fs)),2)/(M*U)

# Convert to dB
Px = 10 * np.log10(Px) - 10*np.log10(L)

generate_plot(x=frequency, y=Px, name='Welch_periodogram_Blackman.pdf')

## Blackman and Tukey
The Blackman and Tukey spectral estimate is also called the correlogram.  This is because it is based on the autocorrelation of the input sequence, and then application of the Wiener-Khintchine theorem to obtain a spectral estimate.  Because the autocorrelation estimate is truncated, a window needs to be applied.  The window should have a non-negative frequency transform.

The Blackman and Tukey estimate is formed as:
$$P_{xx}^{\mathrm{BT}}(f)=\sum_{m=-(M-1)}^{M-1} r_{xx}(m) w(m) e^{-j2\pi f m}$$

In [ ]:
# Start by computing the autocorrelation
rxx = Autocorrelation(x = x, M = M)
# Mirror it and combine to also represent the negative delays
rxx = np.concatenate((np.flip(rxx), rxx[1:]), axis=None)

# Define an index for this
m = np.arange(-M,M+1)

# Define a window - we'll start with the triangular one
w = (M - np.abs(m)) / M

# Use the Wiener Khintchine relationship
Px = 10 * np.log10(np.abs(np.fft.fft(np.multiply(w,rxx),fs)))

generate_plot(x=frequency, y=Px)

Other windows can also be used, for example the Bartlett window:

In [ ]:
w = (M + 1 - np.abs(m)) / (M + 1)

# Use the Wiener Khintchine relationship
Px = 10 * np.log10(np.abs(np.fft.fft(np.multiply(w,rxx),fs)))

generate_plot(x=frequency, y=Px)

or the Parzen window, which has a more complex specification, but applied in exactly the same way:
$$w(n)= \left\{
    \begin{array}{ll}
     1-6\left(\frac{|n-M|}{M+\frac{1}{2}}\right)^2+6\left(\frac{|n-M|}{M+\frac{1}{2}}\right)^3 &; |n-M|\le M/2 \\
      2\left(1-\frac{|n-M|}{M+\frac{1}{2}}\right)^3 &;M/2 < |n-M| \le M
    \end{array}\right.$$

In [ ]:
w_centre = 1 - 6  * (np.abs(m)/(M+0.5))**2 + 6 * (np.abs(m)/(M+0.5))**3
w_outside = 2 * (1-abs(m)/(M+0.5))**3

# Compute the switch points
lower = M - (M // 2)
upper = M + (M // 2)

# Now combine them into one window
w = w_outside
w[lower:upper+1] = w_centre[lower:upper+1]

# Now apply it to the autocorrelation, and compute the results
Px = 10 * np.log10(np.abs(np.fft.fft(np.multiply(w,rxx),fs)))

generate_plot(x=frequency, y=Px)

The Bohman window is defined as:
$$w(n)=\left(1-\left|\frac{n}{M}-1\right|\right)\cos\left(\pi\left|\frac{n}{M}-1\right|\right)+
      \frac{1}{\pi}\sin\left(\pi\left|\frac{n}{M}-1\right|\right)$$

In [ ]:
w = np.multiply((1 - np.abs(m)/M),np.cos(np.pi*np.abs(m)/M)) + np.sin(np.pi*np.abs(m)/M)/np.pi

# Use the Wiener Khintchine relationship
Px = 10 * np.log10(np.abs(np.fft.fft(np.multiply(w,rxx),fs)))

generate_plot(x=frequency, y=Px)

### Minimum variance spectral estimation animation

Compute the autocorrelation, and create the Toeplitz matrix

In [ ]:
rxx = Autocorrelation(x = x, M = p)
R = spl.toeplitz(rxx)

Perform the eigenvalue decomposition

In [ ]:
[d, v] = spl.eig(R)

Invert the elements of the diagonal matrix, and store as a vector

In [ ]:
# eps avoids a divide by zero
U = np.divide(np.ones(p+1), (abs(d))+np.finfo(float).eps)

Transform the eigenvectors. <br>
The result is a matrix of dimensions fs x p - each eigenvector is transformed

In [ ]:
V = abs(np.fft.fft(v.T, fs))**2

Then compute the final spectrum by combining the transformed variables and normalising 
by the length of the correlation vector

In [ ]:
Px_linear = np.divide(p, (np.dot(V.T, U)))

Now plot the final result in a dB scale

In [ ]:
Px = 10*np.log10(Px_linear)

generate_plot(x=frequency, y=Px, name='Minimum_variance_spectral_estimate.pdf')

We can produce an animation of the filter responses as the frequency sweeps from 0 to half the sampling frequency. 
By observing the amplitude of the frequency terms corresponding to the dominant component, the adaptation of the filters to the input signal can be observed.

In [ ]:
#  To compute h_opt, we will need the inverse of R
Rinv = np.linalg.inv(R)

# Define frequencies to be plotted
freq_lower = 0.35
freq_upper = 0.45

Enlarge the figure size and label font size

In [ ]:
pl.rcParams["figure.figsize"] = (16, 8)
pl.rcParams.update({'font.size': 16})

In [ ]:
for freq_sample in range(int(np.ceil(freq_lower*fs)), int(np.floor(freq_upper*fs)+1)):

    # Define y scale limits
    ylim=[-95,10]
    
    # Find the frequency of the sample we are going to consider
    fl = freq_sample / fs

    # And compute its phasor terms
    E = np.exp(1j*2*np.pi*fl*np.arange(0, p+1))

    # Compute the filter coefficients
    h = np.dot(np.dot(Rinv, E), (Px_linear[freq_sample])) / p
    
    # Calculate the frequency response of the filter
    H = np.fft.fft(h.T, fs)
    
    ### Plot it
    # Get current axes instance
    ax = pl.gca()
    # Plot the figure
    ax.plot(frequency, 20*np.log10(abs(H)))
    # Tidy up the x axis and y axis
    ax.set_xlabel('Normalised frequency')
    ax.set_xlim([freq_lower, freq_upper])
    ax.set_ylabel('Magnitude (dB)')
    ax.set_ylim(ylim)
    ax.set_xticks(np.linspace(0.35, 0.45, 11))
    ax.set_yticks(np.linspace(-90, 10, 11))
    ax.set_title('Centre frequency %6.4f' %fl)
    
    # Circle the frequency response amplitude corresponding 
    # to the frequencies of interest
    
    arrowx =(f1-0.35) * 10
    arrowy = (20*np.log10(abs(H[int(f1*fs)]))-ylim[0]) / (ylim[1]-ylim[0])
    if (arrowx>0.01) and (arrowx<0.99) and (arrowy>0.01) and (arrowy<0.99):
        # Create a scale-free ellipse
        el1 = matplotlib.patches.Ellipse(xy = (arrowx, arrowy), 
                                         width = 0.02, height = 0.03,
                                         fc = "none", ec = "black",
                                         transform = ax.transAxes) 
        ax.add_patch(el1)

    arrowx = (f2-0.35) * 10
    arrowy=(20*np.log10(abs(H[int(f2*fs)]))-ylim[0]) / (ylim[1]-ylim[0])
    if (arrowx>0.01) and (arrowx<0.99) and (arrowy>0.01) and (arrowy<0.99):
        el2 = matplotlib.patches.Ellipse(xy = (arrowx, arrowy), 
                                         width = 0.02, height = 0.03,
                                         fc = "none", ec = "black",
                                         transform = ax.transAxes)
        ax.add_patch(el2)
    
    arrowx = (fl-0.35) * 10
    arrowy = (20*np.log10(abs(H[int(fl*fs)]))-ylim[0]) / (ylim[1]-ylim[0])
    if (arrowx>0.01) and (arrowx<0.99) and (arrowy>0.01) and (arrowy<0.99):
        # Add the green small square
        ell = matplotlib.patches.Rectangle(xy = (arrowx-0.01,arrowy-0.02),
                                           width = 0.02, height = 0.04,
                                           fc = "green", ec = "black",
                                           alpha = 0.5, transform = ax.transAxes)
        ax.add_patch(ell)
    
    # And pause briefly so that the graph displays on the screen
    display.clear_output(wait=True)
    pl.pause(0.001)

© The University of Edinburgh: Produced by D. Laurenson, School of Engineering. Initial code conversion by Xing Zixiao.